In [6]:
from datetime import datetime
import numpy as np
import pandas as pd

# Define the custom dtype for the structured array
dtype = np.dtype([
    ('datetime', 'datetime64[m]'),
    ('open', 'f4'),
    ('high', 'f4'),
    ('low', 'f4'),
    ('close', 'f4')
])

# Read the CSV file
df_eurusd = pd.read_csv('./Data/eurusd_2021to2024.csv')

# Convert the datetime column to datetime64
df_eurusd['datetime'] = pd.to_datetime(df_eurusd['datetime'])

# Create the structured array
data_eurusd = np.zeros(len(df_eurusd), dtype=dtype)

# Fill the structured array with data from the dataframe
data_eurusd['datetime'] = df_eurusd['datetime'].values.astype('datetime64[m]')
data_eurusd['open'] = df_eurusd['open'].values
data_eurusd['high'] = df_eurusd['high'].values
data_eurusd['low'] = df_eurusd['low'].values
data_eurusd['close'] = df_eurusd['close'].values


In [7]:
# Read the CSV file
df_btcusd = pd.read_csv('./Data/btcusd_2022to2024.csv')

# Convert the datetime column to datetime64
df_btcusd['datetime'] = pd.to_datetime(df_btcusd['datetime'])

# Create the structured array
data_btcusd = np.zeros(len(df_btcusd), dtype=dtype)

# Fill the structured array with data from the dataframe
data_btcusd['datetime'] = df_btcusd['datetime'].values.astype('datetime64[m]')
data_btcusd['open'] = df_btcusd['open'].values
data_btcusd['high'] = df_btcusd['high'].values
data_btcusd['low'] = df_btcusd['low'].values
data_btcusd['close'] = df_btcusd['close'].values

In [3]:
"""
Klasse für Positionen, soll bei jedem kauf geöffnet werden und bei jedem verkauf geschlossen
"""

class Position:
    def __init__(self, ticker, quantity, price, leverage, margin, type) -> None:
        self.ticker = ticker
        self.quantity = quantity
        self.price = price
        self.total = quantity * price
        self.leverage = leverage
        self.margin = margin
        self.pnl = 0
        self.type = type
        
    def update(self, price, quantity):
        self.price = price
        
    def close(self):
        self = None

In [4]:
from typing import List

class User:
    def __init__(self, name, cash, start_date):
        self.name = name
        self.cash = cash                # Starting cash
        self.start_date = start_date
        self.tickers = ["EUR/USD", "BTC/USD"]
        self.capital_invested = 0
        self.capital = self.cash + self.capital_invested
        self.positions: List[Position] = []
        
    def buy_stock(self, ticker, quantity, price, leverage, type):
        if (ticker in self.tickers) and (quantity*price >= self.cash) and leverage == 1:
            margin = (quantity * price) / leverage
            self.capital_invested += margin
            self.cash -= margin
            self.capital = self.cash + self.capital_invested
            self.positions.append(Position(ticker, quantity, price, leverage, margin, type))
            print(f"{self.name} bought Stock {ticker} with Leverage: {leverage}")

            
        elif ticker not in self.tickers and (quantity*price >= self.cash):
            print("Stock not found")
            
        elif ticker in self.tickers and (quantity*price < self.cash):
            print("Not enough cash")
            
        else:
            print("Stock not found and not enough cash")
        
    def sell_stock(self, postion:Position, quantity):
        if postion.quantity == quantity:
            self.capital_invested -= postion.margin + postion.pnl
            self.cash += postion.margin + postion.pnl
            self.capital = self.cash + self.capital_invested
            self.positions.remove(postion)
        elif postion.quantity > quantity:
            self.capital_invested -= postion.margin + postion.pnl

    def get_cash(self):
        return self.cash

    def get_portfolio(self):
        return self.portfolio

    def get_stock_prices(self):
        return self.stock_prices
    
    def get_data(self, ticker, start_date, end_date):
        if ticker == 'EUR/USD':
            start_date = np.datetime64(start_date, 'm')
            end_date = np.datetime64(end_date, 'm')
            # Create a boolean mask
            mask = (data_eurusd['datetime'] >= start_date) & (data_eurusd['datetime'] <= end_date)
            return data_eurusd[mask]
            
